# Progress reports
Generate a randomly sampled list of team members to do internal progress reports

In [1]:
# Import necessary modules
import requests
from bs4 import BeautifulSoup
import pandas

In [2]:
# Load and parse our team website
page = requests.get("http://www.anaweb.ch/ueber_uns/abteilungen/index_ger.php?key=klin").text
soup = BeautifulSoup(page, 'html.parser')

In [3]:
# Generate dataframe to save the data
Team = pandas.DataFrame()
# Get all names from our page, and save them to a string
Team['Name'] = [str(n.string) for n in soup.find_all(itemprop="name")]
# Drop 'Universität Bern', which we don't want in the list
Team = Team[Team.Name != 'Universität Bern']
# Get them functions as strings
Team['JobTitle'] = [str(i.string) for i in soup.find_all(itemprop="jobTitle")]
# In the first iteration, no one is presenting :)
Team['Presenting'] = False

In [4]:
# Some 'JobTitles' should present, others not
# Based on https://stackoverflow.com/a/13842286/
for index, row in Team.iterrows():
    # Set all 'Doktoranden' and 'Post-Docs' to True
    if 'Do' in row.JobTitle:
        Team.at[index, 'Presenting'] = True
    # Set all 'Assis' and 'Hilfs-Assis' to True   
    if 'ssi' in row.JobTitle:
        Team.at[index, 'Presenting'] = True
    # I guess Valentin should present, too
    if 'Geschäftsführender' in row.JobTitle:
        Team.at[index, 'Presenting'] = True

In [5]:
# These persons don't 'Show & Tell'
print('Out of %s team members, these %s persons *do not* talk '
      'about their work in the lab meeting' %(len(Team),
                                              len(Team[Team['Presenting'] == False])))
Team[Team['Presenting'] == False]

Out of 25 team members, these 8 persons *do not* talk about their work in the lab meeting


,Name,JobTitle,Presenting
2,Johannes Dominik Bastian,Gastarzt aus Orthopädischer Chirurgie und Trau...,False
3,Mathias Bergmann,None,False
4,Nane Boemke,Leiterin der Prosektur und Organisation Klinis...,False
5,Regula Buergy-Meyer,Laborantin,False
9,Hans Grossmann,Präparator,False
15,Adolfo Odriozola,Laborant,False
21,Jeannine Wagner-Zimmermann,Laborantin,False
22,Katrin Weber,None,False


In [6]:
# Put an empty presenting date in a new column
Team['Presented'] = False

In [7]:
# Some team members already presented.
# Let's set their dates, based on https://stackoverflow.com/a/11531402/323100 
Team.loc[Team.Name.str.contains('Cédric'), 'Presented'] = '2018-05-29'
Team.loc[Team.Name.str.contains('Anita'), 'Presented'] = '2018-07-03'
Team.loc[Team.Name.str.contains('Matthias'), 'Presented'] = '2018-09-18'
Team.loc[Team.Name.str.contains('Verdiana'), 'Presented'] = '2018-10-02'

In [8]:
# We already had progress reports of these persons
print('We already had a progress report')
for i, member in Team[Team['Presenting'] &
                      Team['Presented']].sort_values(by = 'Presented').iterrows():
    print('- of',
          member['Name'],
          'on',
          pandas.to_datetime(member['Presented']).strftime('%B %d in %Y'))

We already had a progress report
- of Cédric Zubler on May 29 in 2018
- of Anita Senk on July 03 in 2018
- of Matthias Messerli on September 18 in 2018
- of Verdiana Trappetti on October 02 in 2018


In [9]:
# These persons remain
print('These are the upcoming people in a random but preferred order.')
print('Volunteers will be treated preferentially.')
# Cedric and Anita volunteered to present before I started this notebook.
# Conveniently, 'random_state=12' samples the team member list in a way that
# Cedric and Anita appear last in the list.
# 'frac=1' gives us the full list, so we just iterate through it for nicely
# displaying the output.
# And since Cedric and Anita are last in the list, we just flip the list
# by '.iloc[::-1]' before iteration
for i, member in Team[Team['Presenting'] == True].sample(frac=1,
                                                         random_state=12).iloc[::-1].iterrows():
    if not member['Presented']:
        print('\t-', member['Name'])

These are the upcoming people in a random but preferred order.
Volunteers will be treated preferentially.
	- Helena Röss
	- Sebastian Halm
	- Sara Soltermann
	- Dea Aaldijk
	- Carlos Correa Shokiche
	- Cristian Fernandez Palomo
	- Marine Potez
	- Valentin Djonov
	- Thomas Alexander Gorr
	- Ruslan Hlushchuk
	- Fluri Wieland
	- Oleksiy-Zakhar Khoma
	- David Haberthür


Now send an calendar update to the group with the addresses from the (internal) [Multimail Tool](https://www.anaintra.unibe.ch/multimail.php).